This code is focused on automating the segmentation of nuclei in medical imaging, specifically using a pretrained model (StarDist2D) for image processing. The input to this code is a zip file containing images, which are extracted and preprocessed into TIFF format for further processing. The code normalizes the images, applies the pretrained StarDist2D model to predict instance segmentation (i.e., identifying and delineating nuclei), and saves the segmented output images. The final output consists of processed, segmented images saved in a specified output directory. The program also includes time tracking to measure the performance of processing each image and employs multi-processing to speed up the processing of a large number of images.

In summary:

Input: A zip file with image data (likely in a format like PNG or JPEG), containing medical images requiring nuclei segmentation. <br>
Output: Segmented images with nuclei highlighted, saved as PNG files in an output folder. Additionally, the code uses time tracking and multi-processing for efficient processing of multiple images.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
path_to_zip_file='/content/drive/MyDrive/Dataset_Meningioma Parts/D_2.zip'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall('/content/D_2')

import os
print(len(os.listdir('/content/D_2')))
!pip install stardist
!pip3 install imagecodecs

495
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 13.8 MB/s eta 0:00:00


In [ ]:
import os
import os
import shutil
import cv2
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = 'none'
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible
import time
from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D

In [ ]:
import os
import os
import shutil
import cv2
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
matplotlib.rcParams["image.interpolation"] = 'none'
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible
import time
from stardist import random_label_cmap, _draw_polygons, export_imagej_rois
from stardist.models import StarDist2D

lbl_cmap = random_label_cmap()
files_initial=os.listdir('/content/D_2')
print(files_initial)
print('All Images in Input Folder are',len(files_initial))
for i in range(len(os.listdir('/content/D_2'))):
  start_time=time.time()
  im = cv2.imread('/content/D_2/'+files_initial[i])
  cv2.imwrite('/content/D_2/'+files_initial[i].split('.')[0]+'.tif',im)
  os.remove('/content/D_2/'+files_initial[i])
  end_time=time.time()
  print('Image Number',i)
  print('Total Time taken for Image is ',i,(end_time-start_time))

files=os.listdir('/content/D_2')
print(files)

print('All Images in Input Folder are',len(os.listdir('/content/D_2')))

X = sorted(glob('/content/D_2/*.tif'))
image_names=[]
for i in range(len(sorted(glob('/content/D_2/*.tif')))):
  name=sorted(glob('/content/D_2/*.tif'))[i].split('/')[3].split('.')[0]
  print(name)
  image_names.append(name)
X = list(map(imread,X))
print(image_names)
print(len(X))
print(X)
n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
axis_norm = (0,1)   # normalize channels independently
#axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))

# Load the pretrained model
model = StarDist2D.from_pretrained('2D_versatile_he')

def process_image(i):
    start_time = time.time()
    print('Processing Image Number is', i)

    # Normalize the image
    img = normalize(X[i], 1, 98, axis=axis_norm)

    # Run instance prediction
    labels, _ = model.predict_instances(img, prob_thresh=0.2, nms_thresh=0.2)

    # Display and save segmented image
    plt.imshow(img, clim=(0, 1), cmap='gray')
    plt.imshow(labels, clim=(0, 255), cmap=lbl_cmap, alpha=1)
    plt.axis('off')
    plt.tight_layout()

    # Save the final processed image
    output_folder = '/content/output_images' #New path
    os.makedirs(output_folder, exist_ok=True)

    # Instead of saving a temporary file, we can use in-memory processing
    plt.savefig(os.path.join(output_folder, f'{image_names[i]}.png'), bbox_inches='tight', pad_inches=0)
    plt.close()

    # Read the saved segmented image
    img = cv2.imread(os.path.join(output_folder, f'{image_names[i]}.png'))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    s = hsv[:, :, 2]
    _, thresh = cv2.threshold(s, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    stretch_near = cv2.resize(thresh, (256, 256), interpolation=cv2.INTER_LINEAR)

    # Save the final processed image
    cv2.imwrite(os.path.join(output_folder, f'{image_names[i]}_final.png'), stretch_near)

    end_time = time.time()
    print('Total Time taken for Image Number', i, 'is', (end_time - start_time) * 1000, 'milliseconds')

if __name__ == "__main__":
    # Ensure GPU is utilized for the model
    num_images_to_process = 100  # Process the first 100 images
    with Pool(processes=os.cpu_count()) as pool:
        pool.map(process_image, range(num_images_to_process))


Streaming output truncated to the last 5000 lines.
       [[255, 184, 233],
        [249, 178, 222],
        [238, 166, 212],
        ...,
        [255, 211, 247],
        [243, 201, 237],
        [237, 196, 230]],

       [[255, 183, 232],
        [249, 177, 223],
        [241, 169, 217],
        ...,
        [255, 215, 251],
        [249, 205, 241],
        [244, 200, 235]],

       ...,

       [[249, 189, 227],
        [246, 186, 222],
        [248, 183, 223],
        ...,
        [255, 190, 239],
        [250, 182, 231],
        [245, 177, 226]],

       [[252, 192, 230],
        [243, 185, 223],
        [245, 180, 222],
        ...,
        [253, 187, 233],
        [247, 181, 227],
        [247, 181, 229]],

       [[255, 189, 235],
        [254, 183, 227],
        [252, 176, 223],
        ...,
        [246, 183, 228],
        [242, 181, 225],
        [247, 186, 230]]], dtype=uint8), array([[[235, 168, 212],
        [241, 159, 221],
        [247, 158, 224],
        ...,
        [